In [ ]:
import pandas as pd
import datetime
from sklearn.neural_network import MLPClassifier
import random
import torch
from net import gtnet
from sklearn.metrics import roc_auc_score, recall_score, precision_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
list_users_above_criteria = [
        1032,
        581,
        407,
        290,
        1436,
        1000,
        95,
        1386,
        1431,
        992,
        1717,
        1441,
        122,
        977,
        293,
        1700,
        1744,
        622,

        192,
        1373,
        84,
        1393,
        1432,
        1378,
        225,
        1753,
        2084,
        969,
        280,
        99,
        53,
        983,
        2068,
        193,
        2056,
        2016,
        2109, 
        1995,
        1706,
        2015,
        186,
        137,
        1658,
        2083,
        1383,
        429,
        279]
print(len(list_users_above_criteria))

In [ ]:
seq_len = 14
selected_user = 1032
# 1431 and 95 have bug, excluded for now
# user_lst = [290, 407, 581, 1032, 1436, 1000, 1386]
user_lst = list_users_above_criteria
def match_dataset_date(selected_user, seq_len):
    y_df = pd.read_csv(f'/mnt/results/user_{selected_user}_puqe.csv')
    x_df = pd.read_csv(f'/mnt/results/user_{selected_user}_activity_bodyport_hyperimpute_with_date.csv')
    y_df['date'] = pd.to_datetime(y_df['date'])
    x_df['date'] = pd.to_datetime(x_df['date'])
    dataset = []
    for i in range(len(y_df)):
        row = y_df.iloc[i]
        day_before = row['date'] - datetime.timedelta(days=seq_len)
        x = (x_df[(x_df['date'] > day_before) & (x_df['date'] <= row['date'])])
        dataset.append((x.drop('date', axis=1).to_numpy(), row['answer_text'], selected_user))
    return dataset
# gnn_dataset_1032 = match_dataset_date(selected_user, seq_len)
# gnn_dataset_1032 = list(filter(lambda x: len(x[0])>=seq_len, gnn_dataset))
# baseline_dataset_1032 = list(map(lambda x: (x[0][-7:], x[1]), gnn_dataset))
baseline_ds_lst = []
gnn_ds_lst = []
for u in user_lst:
    temp_dataset = match_dataset_date(u, seq_len)
    temp_dataset = list(filter(lambda x: len(x[0])>=seq_len, temp_dataset))
    gnn_ds_lst.append(temp_dataset)
    baseline_dataset = list(map(lambda x: (x[0][-7:], x[1], x[2]), temp_dataset))
    baseline_ds_lst.append(baseline_dataset)
print(len(baseline_ds_lst))
print(len(gnn_ds_lst))

In [ ]:
# dynamic time warping
from tslearn.metrics import dtw
def get_similarities(target_user, user_lst):
    result = []
    target_user_df = pd.read_csv(f'/mnt/results/user_{target_user}_activity_bodyport_hyperimpute_with_date.csv').drop('date', axis=1)
    for user in user_lst:
        if user == target_user:
            continue
        else:
            curr_user_df = pd.read_csv(f'/mnt/results/user_{user}_activity_bodyport_hyperimpute_with_date.csv').drop('date', axis=1)
            result.append((user, dtw(target_user_df, curr_user_df)))
    return sorted(result, key=lambda x: x[1])
print(get_similarities(186, user_lst))

In [ ]:
# args = {'data': '/mnt/results/user_456546_activity_bodyport_hyperimpute.csv', 'log_interval': 2000, 'optim': 'adam', 'L1Loss': True, 'normalize': 2, 'device': 'cuda:0', 'gcn_true': True, 'buildA_true': True, 'gcn_depth': 2, 'num_nodes': 19, 'dropout': 0.3, 'subgraph_size': 19, 'node_dim': 40, 'dilation_exponential': 2, 'conv_channels': 16, 'residual_channels': 16, 'skip_channels': 32, 'end_channels': 64, 'in_dim': 1, 'seq_in_len': 14, 'seq_out_len': 1, 'horizon': 7, 'layers': 5, 'batch_size': 32, 'lr': 0.0001, 'weight_decay': 1e-05, 'clip': 5, 'propalpha': 0.05, 'tanhalpha': 3, 'epochs': 50, 'num_split': 1, 'step_size': 100}
device = torch.device('cuda:0')
torch.set_num_threads(3)
gnn_raw_copy = gnn_ds_lst.copy()
gnn_ds_lst = []

# user_data format [(X's, y's), ...]
def get_gnn_output(gnn_user_id, user_data):
    model = torch.load(f'/mnt/results/model/model_{gnn_user_id}.pt')
    model.to(device)
    model.eval()
    result = []
    for j in range(len(user_data)):
        # print(d[0].shape)
        curr = user_data[j]
        X = torch.from_numpy(curr[0]).unsqueeze(0).unsqueeze(0).permute(0,1,3,2)
        # print(X.shape)
        X.to(device)
        X = X.type(torch.cuda.FloatTensor)
        out = model(X)
        out = out.squeeze(0).squeeze(0).squeeze(-1)
        result.append((out.detach().cpu().numpy(), curr[1], curr[2]))
    return result

for i in range(len(user_lst)):
    gnn_ds_lst.append(get_gnn_output(user_lst[i], gnn_raw_copy[i]))


In [ ]:
def generate_tsne_2d(gnn_ds_lst):
    all_gnn_data = []
    for user_data in gnn_ds_lst:
        all_gnn_data.extend(user_data)
    gnn_embeddings = list(map(lambda x: np.expand_dims(x[0], axis=0), all_gnn_data))
    print(gnn_embeddings[0].shape)
    class_labels = list(map(lambda x: x[1], all_gnn_data))
    user_labels = list(map(lambda x: x[2], all_gnn_data))
    gnn_embeddings = np.concatenate(gnn_embeddings, axis=0)
    print(gnn_embeddings.shape)
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, random_state=42)
    # tsne_results = tsne.fit_transform(pca_result)
    tsne_results = tsne.fit_transform(gnn_embeddings)

    df_tsne = pd.DataFrame(tsne_results, columns=["X", "Y"])

    df_tsne["user_labels"] = user_labels
    df_tsne["user_labels"] = df_tsne["user_labels"].apply(lambda i: str(i))

    class_map = ['mild', 'medium', 'severe']
    df_tsne["class_labels"] = class_labels
    df_tsne["class_labels"] = df_tsne["class_labels"].apply(lambda i: class_map[i])
    return df_tsne
df_tsne = generate_tsne_2d(gnn_ds_lst)

In [ ]:
markers_dict = {
    'mild': 'o',
    'medium': 'X',
    'severe': '^'

}
plt.figure(figsize=(16,16))

sns.scatterplot(
    x="X", y="Y",
    hue="user_labels",
    style="class_labels",
    data=df_tsne,
    legend="full", s=70,
    alpha=0.9,
    markers=markers_dict
)

In [ ]:
plt.figure(figsize=(8,8))

sns.scatterplot(
    x="X", y="Y",
    hue="user_labels",
    style="class_labels",
    data=df_tsne.loc[(df_tsne['user_labels'] == '1378') | (df_tsne['user_labels'] == '186')],
    legend="full", s=100,
    alpha=0.9,
    markers=markers_dict
)

In [ ]:
plt.figure(figsize=(7,7))

sns.scatterplot(
    x="X", y="Y",
    hue="user_labels",
    style="class_labels",
    data=df_tsne.loc[(df_tsne['user_labels'] == '290') | (df_tsne['user_labels'] == '186')],
    legend="full", s=70,
    alpha=0.9,
    markers=markers_dict
)

In [ ]:
plt.figure(figsize=(4,4))

sns.scatterplot(
    x="X", y="Y",
    hue="user_labels",
    style="class_labels",
    data=df_tsne.loc[(df_tsne['user_labels'] == '983')],
    legend="full", s=100,
    alpha=0.9,
    markers=markers_dict
)

In [ ]:
plt.figure(figsize=(4,4))

sns.scatterplot(
    x="X", y="Y",
    hue="user_labels",
    style="class_labels",
    data=df_tsne.loc[(df_tsne['user_labels'] == '992')],
    legend="full", s=100,
    alpha=0.9,
    markers=markers_dict
)

In [ ]:
# now we use 992's model to generate 983's embedding
left_user = 1378
similar_user = 186
replacement_embedding = get_gnn_output(similar_user, gnn_raw_copy[user_lst.index(left_user)])
replacement_gnn_lst = gnn_ds_lst.copy()
replacement_gnn_lst[user_lst.index(left_user)] = replacement_embedding
replacement_df_tsne = generate_tsne_2d(replacement_gnn_lst)


In [ ]:
print(df_tsne["class_labels"].value_counts())
sns.countplot(x="class_labels",data=df_tsne[df_tsne['user_labels'] == '1000'])
print(df_tsne[df_tsne['user_labels'] == '1000']["class_labels"].value_counts())
#sns.countplot(x="class_labels",data=df_tsne)

In [ ]:
plt.figure(figsize=(7,7))

sns.scatterplot(
    x="X", y="Y",
    hue="user_labels",
    style="class_labels",
    data=replacement_df_tsne.loc[(replacement_df_tsne['user_labels'] == str(left_user)) | (replacement_df_tsne['user_labels'] == str(similar_user))],
    legend="full", s=50,
    alpha=0.9,
    markers=markers_dict
)

In [ ]:
def analysis_result(x, true, clf):
    pred = clf.predict(x)
    recall = recall_score(y_true=true, y_pred=pred, average='micro')
    #roc_auc = roc_auc_score(y_true=gnn_y_test, y_score=gnn_y_pred, average='micro', multi_class='ovr')
    precision = precision_score(y_true=true, y_pred=pred, average='micro')
    print(f"Recall: {recall}")
    #print(f"ROC AUC: {roc_auc}")
    print(f"Precision: {precision}")
    cm = confusion_matrix(y_true=true, y_pred=pred)
    if cm.shape[0] == 2:
        cm_df = pd.DataFrame(cm,
                        index = ['mild','medium'], 
                        columns = ['mild','medium'])
    else:
        cm_df = pd.DataFrame(cm,
                            index = ['mild','medium','severe'], 
                            columns = ['mild','medium','severe'])

    #Plotting the confusion matrix
    plt.figure(figsize=(5,4))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actal Values')
    plt.xlabel('Predicted Values')
    plt.show()
def leave_one_out_validation(target_user, user_lst, gnn_embedding_lst, gnn_raw_lst, baseline_ds_lst):
    user_idx = user_lst.index(target_user)
    gnn_training_set = []
    baseline_training_set = []
    for i in range(len(gnn_embedding_lst)):
        if i == user_idx:
            continue
        else:
            gnn_training_set.extend(gnn_embedding_lst[i])
            baseline_training_set.extend(baseline_ds_lst[i])
    gnn_train_x = list(map(lambda x: x[0], gnn_training_set))
    gnn_train_y = list(map(lambda x: x[1], gnn_training_set))
    baseline_train_x = list(map(lambda x: x[0].ravel(), baseline_training_set))
    baseline_train_y = list(map(lambda x: x[1], baseline_training_set))

    most_similar_user = get_similarities(target_user, user_lst)[0][0]
    gnn_test = get_gnn_output(most_similar_user, gnn_raw_lst[user_idx])
    gnn_test_x = list(map(lambda x: x[0], gnn_test))
    gnn_test_y = list(map(lambda x: x[1], gnn_test))

    baseline_test_x = list(map(lambda x: x[0].ravel(), baseline_ds_lst[user_idx]))
    baseline_test_y = list(map(lambda x: x[1], baseline_ds_lst[user_idx]))
    print("Baseline:")
    base_clf = MLPClassifier(random_state=90, max_iter=300, hidden_layer_sizes=(50, 15,)).fit(baseline_train_x, baseline_train_y)
    print(base_clf.score(baseline_test_x, baseline_test_y))
    analysis_result(baseline_test_x, baseline_test_y, base_clf)
    print("GNN:")
    gnn_clf = MLPClassifier(random_state=90, max_iter=300, hidden_layer_sizes=(40,8,)).fit(gnn_train_x, gnn_train_y)
    print(gnn_clf.score(gnn_test_x, gnn_test_y))
    analysis_result(gnn_test_x, gnn_test_y, gnn_clf)

leave_one_out_validation(290, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(983, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(992, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(1717, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(1032, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(1436, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(290, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
leave_one_out_validation(1386, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)

In [ ]:
# for u in user_lst:
#     print('='*20, u, '='*20)
#     leave_one_out_validation(u, user_lst, gnn_ds_lst, gnn_raw_copy, baseline_ds_lst)